In [1]:
%config Completer.use_jedi = False

In [2]:
import import_ipynb

In [3]:
import argparse, glob, os, torch, warnings, time
from tools import *
from dataLoader import *
from ECAPAModel import ECAPAModel

importing Jupyter notebook from tools.ipynb
importing Jupyter notebook from dataLoader.ipynb
importing Jupyter notebook from ECAPAModel.ipynb
importing Jupyter notebook from loss.ipynb
importing Jupyter notebook from model.ipynb


In [4]:
import sys
import torch
from torch.utils.data import dataloader
from torch.multiprocessing import reductions
from multiprocessing.reduction import ForkingPickler

default_collate_func = dataloader.default_collate


def default_collate_override(batch):
    dataloader._use_shared_memory = False
    return default_collate_func(batch)

setattr(dataloader, 'default_collate', default_collate_override)

for t in torch._storage_classes:
    if sys.version_info[0] == 2:
        if t in ForkingPickler.dispatch:
            del ForkingPickler.dispatch[t]
    else:
        if t in ForkingPickler._extra_reducers:
            del ForkingPickler._extra_reducers[t]

### ECAPA + ElasticLoss (All)

In [ ]:
parser = argparse.ArgumentParser(description = "ECAPA_trainer")
## Training Settings
parser.add_argument('--num_frames', type=int,   default=200,     help='Duration of the input segments, eg: 200 for 2 second')
parser.add_argument('--max_epoch',  type=int,   default=80,      help='Maximum number of epochs')
parser.add_argument('--batch_size', type=int,   default=400,     help='Batch size')
parser.add_argument('--n_cpu',      type=int,   default=4,       help='Number of loader threads')
parser.add_argument('--test_step',  type=int,   default=1,       help='Test and save every [test_step] epochs')
parser.add_argument('--lr',         type=float, default=0.0001,   help='Learning rate')
parser.add_argument("--lr_decay",   type=float, default=1,    help='Learning rate decay every [test_step] epochs')

## Training and evaluation path/lists, save path

parser.add_argument('--train_list', type=str,   default="/data/vox/vox_train_list.txt",     help='The path of the training list, https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/train_list.txt')
parser.add_argument('--train_path', type=str,   default="/data/vox/dev/aac",                    help='The path of the training data, eg:"/data08/VoxCeleb2/train/wav" in my case')
parser.add_argument('--eval_list',  type=str,   default="/data/vox/voxceleb1_test.txt",              help='The path of the evaluation list, veri_test2.txt comes from https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test2.txt')
parser.add_argument('--eval_path',  type=str,   default="/data/vox1_test/wav",                    help='The path of the evaluation data, eg:"/data08/VoxCeleb1/test/wav" in my case')
parser.add_argument('--musan_path', type=str,   default="/data/musan",                    help='The path to the MUSAN set, eg:"/data08/Others/musan_split" in my case')
parser.add_argument('--rir_path',   type=str,   default="/data/rir/simulated_rirs",     help='The path to the RIR set, eg:"/data08/Others/RIRS_NOISES/simulated_rirs" in my case');
parser.add_argument('--save_path',  type=str,   default="/data/vox",                                     help='Path to save the score.txt and models')
parser.add_argument('--model_name', type=str,   default='resnetse_softmax',                        help='Model name to use')
parser.add_argument('--encoder', type=str,   default='resnetse',                        help='Loss to use')
parser.add_argument('--loss_name', type=str,   default='softmax',                        help='Loss to use')
parser.add_argument('--initial_model',  type=str,   default="",                                    help='Path of the initial_model')

## Model and Loss settings
parser.add_argument('--C',       type=int,   default=1024,   help='Channel size for the speaker encoder')
parser.add_argument('--m',       type=float, default=0.2,    help='Loss margin in AAM softmax')
parser.add_argument('--std',     type=float, default=0.05,    help='Loss sd in AAM softmax')
parser.add_argument('--plus',     type=float, default=False,    help='Sort random value from gaussian')
parser.add_argument('--s',       type=float, default=30,     help='Loss scale in AAM softmax')
parser.add_argument('--n_class', type=int,   default=5994,   help='Number of speakers')

## Command
parser.add_argument('--eval',    dest='eval', action='store_true', help='Only do evaluation')

## Initialization
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args, _ = parser.parse_known_args()
args = init_args(args)

## Define the data loader
trainloader = train_loader(**vars(args))
trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = args.batch_size, shuffle = True, num_workers = args.n_cpu, drop_last = True)

## Search for the exist models
modelfiles = glob.glob('%s/model_0*.model'%args.model_save_path)
modelfiles.sort()

## Only do evaluation, the initial_model is necessary
if args.eval == True:
    s = ECAPAModel(**vars(args))
    print("Model %s loaded from previous state!"%args.initial_model)
    s.load_parameters(args.initial_model)
    EER, minDCF = s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)
    print("EER %2.2f%%, minDCF %.4f%%"%(EER, minDCF))
    quit()

## If initial_model is exist, system will train from the initial_model
if args.initial_model != "":
    print("Model %s loaded from previous state!"%args.initial_model)
    s = ECAPAModel(**vars(args))
    s.load_parameters(args.initial_model)
    epoch = 1

## Otherwise, system will try to start from the saved model&epoch
elif len(modelfiles) >= 1:
    print("Model %s loaded from previous state!"%modelfiles[-1])
    epoch = int(os.path.splitext(os.path.basename(modelfiles[-1]))[0][6:]) + 1
    s = ECAPAModel(**vars(args))
    s.load_parameters(modelfiles[-1])
## Otherwise, system will train from scratch
else:
    epoch = 1
    s = ECAPAModel(**vars(args))

EERs = []
score_file = open(args.score_save_path, "a+")

while(1):
    ## Training for one epoch
    loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

    ## Evaluation every [test_step] epochs
    if epoch % args.test_step == 0:
        s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
        EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
        print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
        score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
        score_file.flush()

    if epoch >= args.max_epoch:
        quit()

    epoch += 1

08-29 12:38:06 Model para number = 1.33


  0%|          | 0/4715 [00:00<?, ?it/s]ng: 100.00%,  Loss: 7.58502, ACC: 0.50% 

100%|██████████| 4715/4715 [01:23<00:00, 56.30it/s]


2023-08-29 13:05:03 1 epoch, ACC 0.50%, EER 21.53%, bestEER 21.53%


  0%|          | 0/4715 [00:00<?, ?it/s]ng: 100.00%,  Loss: 6.24959, ACC: 3.79% 

...

100%|██████████| 4715/4715 [01:21<00:00, 58.14it/s]


2023-08-29 17:08:45 10 epoch, ACC 41.29%, EER 7.91%, bestEER 7.91%


  0%|          | 0/4715 [00:00<?, ?it/s]ng: 100.00%,  Loss: 2.71450, ACC: 44.00% 

100%|██████████| 4715/4715 [01:18<00:00, 59.89it/s]


2023-08-29 17:35:41 11 epoch, ACC 44.00%, EER 7.69%, bestEER 7.69%
